In [1]:
import os
import json
import pandas as pd

In [2]:
os.getcwd()

'/home/shachar/Desktop/PythonProjects/Lab/MVTrials/reg_model'

In [4]:
grid_res_dir = 'grid_search_results'

In [6]:
dataset_grid_res = {'_'.join(x.split('_')[:-2]): x for x in os.listdir(grid_res_dir) if
                    x.endswith('.csv') and 'process' not in x}
dataset_grid_res

{'rmftsa_ctoarrivals': 'rmftsa_ctoarrivals_grid_val.csv',
 'accent': 'accent_grid_val.csv',
 'analcatdata_germangss': 'analcatdata_germangss_grid_val.csv',
 'analcatdata_vineyard': 'analcatdata_vineyard_grid_val.csv',
 'ecoli': 'ecoli_grid_val.csv',
 'blood_transfusion': 'blood_transfusion_grid_val.csv',
 'credit_approval': 'credit_approval_grid_val.csv',
 'diabetes': 'diabetes_grid_val.csv',
 'spam': 'spam_grid_val.csv',
 'wifi': 'wifi_grid_val.csv',
 'sonar': 'sonar_grid_val.csv',
 'banknote': 'banknote_grid_val.csv',
 'iris': 'iris_grid_val.csv',
 'tokyo': 'tokyo_grid_val.csv',
 'credit_g': 'credit_g_grid_val.csv',
 'parkinson': 'parkinson_grid_val.csv',
 'arsenic_male_bladder': 'arsenic_male_bladder_grid_val.csv',
 'breast': 'breast_grid_val.csv',
 'climate': 'climate_grid_val.csv',
 'wine': 'wine_grid_val.csv'}

In [12]:
os.makedirs("param_files/csv", exist_ok=True)
os.makedirs("param_files/json", exist_ok=True)

In [13]:
df = pd.read_csv(os.path.join(grid_res_dir, list(dataset_grid_res.values())[0]))
df.head()

,dataset,conf_num,reg_type,weight_type,alpha,use_layer_norm,use_aug,lr,batch_size,train_full_score,train_partial_score,val_full_score,val_partial_score
0,Rmftsa_ctoarrivals,1,l1,None,0.5,False,False,0.01,32,0.964286,0.932 +- 0.059,0.906977,0.891 +- 0.055
1,Rmftsa_ctoarrivals,2,l1,None,0.5,False,False,0.01,64,0.970238,0.928 +- 0.062,0.953488,0.891 +- 0.063
2,Rmftsa_ctoarrivals,3,l1,None,0.5,False,False,0.10,32,0.940476,0.908 +- 0.056,0.930233,0.894 +- 0.068
3,Rmftsa_ctoarrivals,4,l1,None,0.5,False,False,0.10,64,0.916667,0.903 +- 0.058,0.883721,0.877 +- 0.055
4,Rmftsa_ctoarrivals,5,l1,None,0.5,False,True,0.01,32,0.946429,0.912 +- 0.056,0.953488,0.918 +- 0.06


In [14]:
for dataset, grid_res in dataset_grid_res.items():
    df = pd.read_csv(os.path.join('grid_search_results', grid_res))
    df["train_partial_scores_mean"] = df["train_partial_score"].apply(lambda x: float(x.split(" +- ")[0]))
    df["val_partial_scores_mean"] = df["val_partial_score"].apply(lambda x: float(x.split(" +- ")[0]))

    # df["full_score"] = df[["train_full_score", "val_full_score"]].min(axis=1)
    # df["partial_scores_mean"] = df[["train_partial_scores_mean", "val_partial_scores_mean"]].min(axis=1)

    df["train_diff_full_partial"] = df["train_full_score"] - df["train_partial_scores_mean"]
    df["val_diff_full_partial"] = df["val_full_score"] - df["val_partial_scores_mean"]

    df["diff_full_partial_min"] = df[["train_diff_full_partial", "val_diff_full_partial"]].min(axis=1)
    df["partial_scores_mean_min"] = df[["train_partial_scores_mean", "val_partial_scores_mean"]].min(axis=1)

    df.sort_values(by="diff_full_partial_min", ascending=True, inplace=True, kind='stable')
    df.sort_values(by="partial_scores_mean_min", ascending=False, inplace=True, kind='stable')

    # df.sort_values(by="val_diff_full_partial", ascending=True, inplace=True, kind='stable')
    # df.sort_values(by="val_partial_scores_mean", ascending=False, inplace=True, kind='stable')
    df.to_csv(os.path.join("param_files", "csv", f"{dataset}_params.csv"), index=False)

    # create params dict of reg_type	weight_type	alpha	use_layer_norm	use_aug	lr	batch_size

    n_best_samples = 10

    params = {
        "reg_type": str(df.iloc[:n_best_samples]["reg_type"].mode().tolist()[0]),
        "weight_type": str(df.iloc[:n_best_samples]["weight_type"].mode().tolist()[0]),
        "alpha": float(df.iloc[:n_best_samples]["alpha"].mode().tolist()[0]),
        "use_layer_norm": df.iloc[:n_best_samples]["use_layer_norm"].mode().tolist()[0],
        # "use_aug": bool(df.iloc[0]["use_aug"]),
        "use_aug": df.iloc[:n_best_samples]["use_aug"].mode().tolist()[0],
        "lr": float(df.iloc[:n_best_samples]["lr"].mode().tolist()[0]),
        "batch_size": int(df.iloc[:n_best_samples]["batch_size"].mode().tolist()[0]),
    }

    with open(os.path.join("param_files", "json", f"{dataset}_params.json"), 'w') as f:
        json.dump(params, f)


In [15]:
df.iloc[:5]["use_layer_norm"].mode().tolist()[0]

False

In [31]:
if os.path.isfile("removal_test_commands"):
    os.remove("removal_test_commands")

for i, dataset in enumerate(os.listdir("param_files/json")):
    f = open("removal_test_commands", 'w' if i == 0 else 'a')
    dataset = "_".join(dataset.split("_")[:-1])
    f.write(f"python main_experimens.py --dataset {dataset.capitalize()} --run_grid False --run_ts False --full_test True\n")

f.close()

# show commands
with open("removal_test_commands", 'r') as f:
    print(f.read())

python main_experimens.py --dataset Blood_transfusion --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Ecoli --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Arsenic_male_bladder --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Tokyo --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Spam --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Analcatdata_vineyard --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Diabetes --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Sonar --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Climate --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Breast --run_grid False --run_ts False --full_test True
python main_experimens.py --dataset Banknot

In [29]:
# load all params files to one df

df = pd.DataFrame()

for dataset in os.listdir("param_files/json"):
    if dataset.endswith(".json"):
        dataset = "_".join(dataset.split("_")[:-1])

        if dataset.lower() in ['wifi', 'wine']:
            continue

        with open(os.path.join("param_files", "json", dataset + "_params.json"), 'r') as f:
            params = json.load(f)
        
        df = df.append(pd.DataFrame(params, index=[dataset]))

df.tail(10)

,reg_type,weight_type,alpha,use_layer_norm,use_aug,lr,batch_size
climate,l2,avg,0.5,False,False,0.010,32
breast,l1,None,0.0,False,False,0.010,32
banknote,l2,loss,0.5,False,False,0.100,64
iris,l2,mult,1.0,True,False,0.100,32
parkinson,var,avg,1.0,False,True,0.001,64
accent,var,avg,0.5,False,True,0.010,64
credit_g,max,None,0.5,False,False,0.010,32
credit_approval,l2,None,0.5,False,True,0.100,32
analcatdata_germangss,l1,avg,0.5,False,True,0.010,32
rmftsa_ctoarrivals,l1,None,0.5,False,False,0.010,32
